In [1]:
from datasets import load_from_disk, load_dataset
from datasets import concatenate_datasets

raw_re_dataset = load_from_disk('../datasets/base/relation_extraction_raw')
raw_re_dataset_gpt4 = load_from_disk('../datasets/base/relation_extraction_gpt4_raw')
raw_re_dataset_gpt4_2 = load_from_disk('../datasets/base/relation_extraction_gpt4_raw_2')
raw_re_dataset['train'] = concatenate_datasets([raw_re_dataset['train'], raw_re_dataset_gpt4['train'], raw_re_dataset_gpt4_2['train']])
raw_re_dataset['test'] = concatenate_datasets([raw_re_dataset['test'], raw_re_dataset_gpt4['test'], raw_re_dataset_gpt4_2['test']])
raw_re_dataset['validation'] = concatenate_datasets([raw_re_dataset['validation'], raw_re_dataset_gpt4['validation'], raw_re_dataset_gpt4_2['validation']])

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
raw_re_dataset

DatasetDict({
    validation: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 106
    })
    test: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 106
    })
    train: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 1049
    })
})

In [3]:
filter_relations = ['kapcsolat', 'versenyfelügyeleti', 'okoz', 'frontol', 'kapott', 'elleni össztűz', 'intézet', 'bíró', 'ár', 'ügyvéd', 'terület', 'kapcsolódik', 'köthető', 'érkezik', 'Nincs kapcsolat.', 'fordul', 'származás', 'adatigénylő - adatok kiadásáért felelős hatóság', 'büntetésben részesül', 'hivatalos', 'győztes', 'hierarchikus', 'megbíráltatás', 'ellentétes vélemények', 'köti']

In [4]:
def fix_relation(row):
    return {'relations': [r for r in row['relations'] if (r['object'][0].isupper() and r['subject'][0].isupper() and r['relation'] not in filter_relations)]}

In [5]:
raw_re_dataset = raw_re_dataset.map(fix_relation)

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/1049 [00:00<?, ? examples/s]

In [7]:
raw_re_dataset['train'][5]['paragraph']

'Közben, március 23. és április 6. között ment a téli rezsicsökkentés reklámozása 920 millió forintért. Orbán Viktor még március 7-én ígért minden háztartásnak 12 ezer forint jóváírást a gázszámlából, és ezt 900 tv-spotban reklámozták a TV2 csatornáin a Magno Studio Kft.-n keresztül.'

In [8]:
from collections import Counter
allso = []
cnt = Counter()

for row in raw_re_dataset['train']:
    for relation in row['relations']:
        allso.append(relation['object'])
        allso.append(relation['subject'])
        
allrel = []
for split in raw_re_dataset:
    for row in raw_re_dataset[split]:
        for relation in row['relations']:
            allrel.append(relation['relation'])
            cnt[relation['relation']] += 1

In [9]:
raw_re_dataset.filter(lambda row: len(row['relations']) > 0)

Filter:   0%|          | 0/106 [00:00<?, ? examples/s]

Filter:   0%|          | 0/106 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1049 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 73
    })
    test: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 71
    })
    train: Dataset({
        features: ['paragraph', 'index', 'prompt', 'output', 'relations'],
        num_rows: 720
    })
})

In [10]:
cnt.most_common(50)

[('tulajdonos', 109),
 ('üzleti', 60),
 ('vezető', 34),
 ('elnök', 25),
 ('együttműködés', 14),
 ('vezérigazgató', 13),
 ('üzleti kapcsolat', 12),
 ('kommunikációs', 12),
 ('pénzügyi', 12),
 ('jogi', 12),
 ('támogató', 11),
 ('ellenőrzés', 11),
 ('képviselő', 10),
 ('munkavállaló', 10),
 ('kommunikáció', 10),
 ('politikai', 9),
 ('munkakapcsolat', 9),
 ('házastárs', 9),
 ('munkatárs', 9),
 ('alkotó', 9),
 ('vizsgálat', 8),
 ('konfliktus', 8),
 ('szerződéses', 8),
 ('igazgató', 8),
 ('üzleti partner', 7),
 ('párttag', 7),
 ('alapító', 7),
 ('érdekeltség', 6),
 ('támogatásban részesül', 6),
 ('jelölt', 6),
 ('tulajdonos vagy vezető', 6),
 ('szerződés', 5),
 ('igazgatósági tag', 5),
 ('polgármester', 5),
 ('alelnök', 5),
 ('személyes', 5),
 ('frakcióvezető', 5),
 ('megbízás', 5),
 ('nyomozás', 5),
 ('ügyfél', 5),
 ('jogi eljárás', 5),
 ('vezetői', 5),
 ('főigazgató', 5),
 ('kölcsönadó', 4),
 ('vásárló', 4),
 ('tanácsadó', 4),
 ('szóvivő', 4),
 ('információcsere', 4),
 ('ügyvezető', 4),
 (

In [11]:
def format_input(paragraph):
    text = ''
    text += 'Bekezdés:\n"' + paragraph.replace('"', "'") + '"\n'
    return text.strip()+'\n\nRelációk:\n'

def format_output(relation):
    text = ''
    for rel in relation['relations']:
        text += 'Indoklás: "' + rel['explanation'] + '"\n'
        text += 'Kapcsolat: "' + rel['relation'] + '"\n'
        text += 'Tárgy: "' + rel['subject'] + '"\n'
        text += 'Alany: "' + rel['object'] + '"\n\n'
    if relation['relations'] == []:
        text += 'Nincs kapcsolat.\n\n'
    return text.strip()

In [12]:
re_dataset = raw_re_dataset.map(lambda row: {'input': format_input(row['paragraph']), 'output': format_output(row)})

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/1049 [00:00<?, ? examples/s]

In [13]:
print(re_dataset['train'][4]['input'])

Bekezdés:
"Tavaly szeptember óta a BKV összesen 23 céggel, közöttük bétékkel, offshore cégekkel, ügyvédi irodákkal kötött tanácsadói szerződést, ezeknek az összértéke meghaladja a százmillió forintot. A szerződő cégek között van - a Dandy Trading Ltd. és az Everest Capita S.A. mellett - az Expert Kft., a Fazeno Bt., a Felsman Balázs volt gazdasági minisztériumi államtitkár által is fémjelzett Force Motrice Zrt., a V-Valeur Kft. és a Noguchi PR-ügynökség. A legnagyobb értékű szerződéseket a Dandy Tradinggel és az Everest Capital S. A-val kötötték, közbeszerzés és versenyeztetés kiírása nélkül."

Relációk:



In [14]:
re_dataset.remove_columns(['paragraph', 'index', 'prompt', 'relations']).save_to_disk('../datasets/tasks/relation_extraction_io2')

Saving the dataset (0/1 shards):   0%|          | 0/106 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/106 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1049 [00:00<?, ? examples/s]

In [15]:
re_dataset.save_to_disk('../datasets/tasks/relation_extraction')

Saving the dataset (0/1 shards):   0%|          | 0/106 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/106 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1049 [00:00<?, ? examples/s]

In [16]:
from datasets import load_from_disk
re_dataset = load_from_disk('../datasets/tasks/relation_extraction_io2')

In [17]:
re_dataset['train'][-1]

{'output': 'Indoklás: "A szöveg szerint Vitézy Dávid korábban BKK vezető volt, de a problémákat a távozása utáni vezetés idézte elő."\nKapcsolat: "előző vezető"\nTárgy: "Vitézy Dávid"\nAlany: "BKK"\n\nIndoklás: "A szöveg alapján Dabóczi Kálmán a BKK akkori vezérigazgatójaként írta alá azt a szerződésmódosítást, amely problémákat okozott a BKK-nál."\nKapcsolat: "vezérigazgató"\nTárgy: "Dabóczi Kálmán"\nAlany: "BKK"\n\nIndoklás: "A szöveg szerint Tarlós István, mint korábbi főpolgármester, büntetőfeljelentést tett az ügyben."\nKapcsolat: "büntetőfeljelentést tevő"\nTárgy: "Tarlós István"\nAlany: "BKK ügy"',
 'input': 'Bekezdés:\n"Vitézy Dávid azonban már korábban is jelezte, hogy a fő problémát a távozása utáni BKK-vezetés idézte elő. Cikkünkre reagálva elmondta, hogy az a szerződésmódosítás, mely lehetővé tette a vállalkozónak, hogy nem igazoltan működőképes, nem tesztelt eszközökért is számlázzon, 2015 nyarán született, Dabóczi Kálmán akkori vezérigazgató írta alá. Ennek a módosításnak